In [1]:
import numpy as np
import pandas as pd
import os
import subprocess
import sys
import argparse
import glob
import requests
import astropy.units as u
from astropy.io import fits as pyfits
import matplotlib.pyplot as plt
from urllib.request import urlopen
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.time import Time
from jinja2 import Environment, FileSystemLoader
from PIL import Image, ImageDraw, ImageFont

C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\Jack Pope\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Importing File and Saving

In [2]:
url = "https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=19.0&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&format=csv"

csv_filename = 'ztf_transients.csv'

response = requests.get(url)
if response.status_code == 200:
    with open(csv_filename, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded {csv_filename}")

transients = pd.read_csv("ztf_transients.csv")

num_images = 6

Downloaded ztf_transients.csv


### Get List of Transient Names

In [3]:

num_instances = len(transients['ZTFID'])
nested_list = []

for i in range(num_instances):
    row = []
    for key in transients:
        row.append(transients[key][i])
    nested_list.append(row) 

for i in range(2):
    object = nested_list[i]
    name = object[0]
    ra = object[2]
    dec = object[3]
    skyguy = SkyCoord(ra = ra, dec = dec, unit=(u.hourangle, u.deg))    

nested_list
transients

,ZTFID,IAUID,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,b,A_V
0,ZTF17aaapufz,AT2016blu,12:35:52.28,+27:55:55.4,1726.75,g,16.4333,-13.87,>2.09,2.09,>0,LBV,0.00261,86.456839,0.048
1,ZTF17aaazdba,AT2019azh,08:13:16.95,+22:38:53.9,561.73,g,15.2769,-19.76,66.42,24.318,42.102,TDE,0.022,27.562336,0.122
2,ZTF17aabtvsy,SN2022yei,10:35:32.09,+37:38:59.0,1870.99,r,18.0303,-19.41,>34.229,>6.01,28.219,SN Ia,0.06922,59.641962,0.053
3,ZTF17aacldgo,SN2022zxv,03:09:24.35,-04:53:39.2,1897.75,g,18.7979,-18.91,>8.87,>3.85,>5.02,SN Ia,0.072,-50.332472,0.183
4,ZTF17aacpbmv,SN2023wtm,03:17:51.82,-00:06:17.8,2261.83,g,17.7727,-17.45,21.11,7.373,13.737,SN Ic,0.023,-45.665483,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8846,ZTF24aahlvgk,SN2024ewu,07:01:04.26,+24:23:22.0,2392.70,g,18.4201,-19.49,>17.897,>2.01,15.887,SN Ia-pec,0.08,12.847346,0.162
8847,ZTF24aahmgvr,SN2024exp,09:44:01.40,-14:54:05.2,2403.75,r,17.1213,-18.65,25.135,8.305,16.83,SN Ia,0.03,28.092871,0.249
8848,ZTF24aahmqzq,SN2024ead,15:47:50.73,-08:25:59.0,2390.98,g,18.2567,-19.26,>10.095,>0.09,>10.005,SN Ia,0.05931,34.424666,0.381
8849,ZTF24aahoihk,SN2024evm,07:52:18.48,+18:08:08.3,2399.65,g,17.4437,-19.41,>20.692,>7.96,12.732,SN Ia,0.05,21.342204,0.142


In [4]:
from astropy.coordinates import SkyCoord
from ztfquery.utils import stamps
import io

def plot_ls_cutout(ddir, name, c):
    """ Plot cutout from Legacy Survey """

    ra = c.ra.deg
    dec = c.dec.deg

    if dec < 0:
        decsign = "-"
    else: 
        decsign = "+"

    fname = ddir + "\\%s_legsurv.png"%name
    if os.path.isfile(fname)==False:
        url = "http://legacysurvey.org/viewer/cutout.jpg?ra=%s&dec=%s&layer=ls-dr9&pixscale=0.05&bands=grz" %(ra,dec)
        plt.figure(figsize=(2.1,2.1), dpi=120)
        try:
            r = requests.get(url)
            plt.imshow(Image.open(io.BytesIO(r.content)))
            plt.title("LegSurv DR9", fontsize = 12)
            plt.axis('off')
            plt.tight_layout()
            plt.savefig(fname, bbox_inches="tight")
        except Exception as e:
            # Print the exception to understand what went wrong
            print(f"An error occurred: {e}")
            return None
        # you want to save it anyway so you don't do this over and over again
        plt.close()
        
    return fname

In [5]:
def plot_ps1_cutout(ddir, name, c, arcsec_width=12):
    """
    Plot cutout from Pan-STARRS.

    Parameters:
    - ddir: Directory to save the image.
    - name: Name for the image file.
    - ra: Right ascension of the target.
    - dec: Declination of the target.
    - arcsec_width: Desired width of the image in arcseconds (default is 12 arcseconds).
    """
    ra = c.ra.deg
    dec = c.dec.deg
    
    # Ensure the directory exists
    if not os.path.exists(ddir):
        os.makedirs(ddir)
    
    # Determine the number of pixels corresponding to the desired arcsecond width
    pixels = int((arcsec_width / 60) * 240)
    
    # Determine the filename for the cutout image
    fname = os.path.join(ddir, f"{name}_ps1.png")
    
    # Check if the file already exists
    if not os.path.isfile(fname):
        try:
            # Get the Pan-STARRS stamp image
            img = stamps.get_ps_stamp(ra, dec, size=pixels, color=["y", "g", "i"])
            
            # Plot and save the image
            plt.figure(figsize=(2.1, 2.1), dpi=120)
            plt.imshow(np.asarray(img))
            plt.title("PS1 (y/g/i)", fontsize=12)
            plt.axis('off')
            plt.tight_layout()
            plt.savefig(fname, bbox_inches="tight")
            plt.close()
        except Exception as e:
            print(f"Failed to fetch or save Pan-STARRS image for {name}: {e}")
            return None
    
    return fname

### Generating a Row of Images

In [6]:
import vlass_search
from urllib.error import HTTPError

def generate_row(name, c, image_dir='images'):
    """
    Generates a row of VLASS PNGs at a given sky coordinate

    Parameters
    -----------
    name: name of the source
    c: coordinates as SkyCoord object
    -----------
    """
    try:
        images, epochs, dates = vlass_search.run_search(name, c)
    except HTTPError as e:
        if e.code == 404:
            print(f"Error 404: URL not found for {name}")
            images, epochs, dates = ['images/unimaged.png'] * 3, ['NA'] * 3, ['Invalid date'] * 3
        else:
            raise e
    
    # Append LS and Pan-STARRS images
    images.append(plot_ls_cutout(image_dir, name, c))
    images.append(plot_ps1_cutout(image_dir, name, c))
    
    return {'name': name, 'images': images, 'epochs': epochs, 'vla_dates': dates}



File downloaded to: VLASS_dyn_summary.php


In [7]:
def get_multiple_rows(start, end):

    image_paths = []
    counter = 1

    for i in range(start - 1, end):
        name = transients['ZTFID'][i]
        ra = transients['RA'][i]
        dec = transients['Dec'][i]
        ztf_date = transients['peakt'][i]
        skyguy = SkyCoord(ra = ra, dec = dec, unit = (u.hourangle, u.deg))
        # add date to dictionary items
        object_dict = generate_row(name, skyguy)
        object_dict['ztf_date'] = ztf_date
        image_paths.append(object_dict)
        print(f"Object #{counter} complete!")
        counter += 1

    for obj in image_paths:
        deltas = []
        print(obj)

        for x in obj['vla_dates']:
            # Convert VLA dates to Julian dates
            if x != 'Invalid date':
                x = Time(x, format = 'iso')
                vla_jd_date = x.jd
            else:
                vla_jd_date = 'NA'
            deltas.append(vla_jd_date)
            print(f"VLA JD Date: {vla_jd_date} (type: {type(vla_jd_date)})")

        obj['ztf_date'] += 2458000
        print(f"ZTF JD Date: {obj['ztf_date']} (type: {type(obj['ztf_date'])})")

        # Ensure the type of ZTF date is float
        ztf_jd_date = float(obj['ztf_date'])

        for i in range(len(deltas)):
            if deltas[i] != 'NA':
                deltas[i] = round(deltas[i] - ztf_jd_date, 2)

        obj['delta_ts'] = deltas
        print(f"Deltas: {deltas}")
        
    return image_paths

In [8]:
from PIL import Image, ImageDraw, ImageFont

def create_unimaged_image(save_path):
    # Define image size and colors
    width, height = 400, 400
    background_color = (250, 150, 150)  # Black background
    text_color = (255, 255, 255)  # White text

    # Create a new image with the specified background color
    image = Image.new('RGB', (width, height), color=background_color)

    # Get a drawing context
    draw = ImageDraw.Draw(image)

    # Load a font
    try:
        font = ImageFont.truetype("RobotoSlab-Regular.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Define the text and get its size
    text = "Unimaged"
    text_width, text_height = draw.textsize(text, font=font)

    # Calculate the position for the text to be centered
    text_x = (width - text_width) / 2
    text_y = (height - text_height) / 2

    # Add the text to the image
    draw.text((text_x, text_y), text, fill=text_color, font=font)

    # Save the image
    image.save(save_path)
    print(f"Image saved as {save_path}")

# Example usage
create_unimaged_image("images\\unimaged.png")


Image saved as images\unimaged.png


C:\Users\Jack Pope\AppData\Local\Temp\ipykernel_53612\1552530876.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


### Create Page

In [89]:
template_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Image Gallery</title>
    <link rel="stylesheet" href="styles.css">     
</head>
<body>
    <div class="container">
        <h1>Image Gallery</h1>
        <div class="gallery-header">
            <div class="header-item">Epoch 1</div>
            <div class="header-item">Epoch 2</div>
            <div class="header-item">Epoch 3</div>
            <div class="header-item">Legacy Survey Image</div>
            <div class="header-item">Panstarrs Image</div>
        </div>
        {% for row in images %}
        <div class="row">
            <div class="name">{{ row['name'] }}</div>
            <div class="image-row">
                {% for i in range(5) %}
                    <div class="image-container">
                        <img src="{{ row['images'][i] }}" alt="Image" class="gallery-image">
                        {% if i < 3 %}
                            {% if row['delta_ts'][i] != 'NA' %}
                                <p class="delta-t">{{ row['delta_ts'][i] }} days</p>
                            {% else %}
                                <p class="delta-t">NA</p>
                            {% endif %}
                        {% endif %}
                    </div>
                {% endfor %}
            </div>
        </div>
        {% endfor %}
    </div>
</body>
</html>
"""


In [91]:
def start_html_page(object_names, template=template_content):
    # Set up the Jinja2 environment
    env = Environment(loader=FileSystemLoader('.'))

    # Create a Jinja2 template from the content
    template = env.from_string(template_content)

    # List to store rows for each object
    rows = []

    for name in object_names['ZTFID']:
        # Generate a row structure with placeholders
        row = {
            'name': name,
            'images': ['images/placeholder.png'] * 5,  # Placeholder images, adjust as needed
            'delta_ts': ['NA'] * 3  # Placeholder delta times, adjust as needed
        }
        rows.append(row)

    # Render the template with the placeholder rows
    html_content = template.render(images=rows)

    # Define the path to save the HTML file
    html_filepath = 'what.html'

    # Write the rendered HTML content to the file
    with open(html_filepath, 'w') as file:
        file.write(html_content)

    print(f"HTML page generated and saved as {html_filepath}")

# Example usage
start_html_page(transients)

HTML page generated and saved as what.html


In [93]:
from bs4 import BeautifulSoup

# Function to update the HTML file with actual images and delta times
def update_html_page(start_row, end_row, html_filepath='ooga_booga.html'):
    # Generate the actual image data
    image_paths = get_multiple_rows(start_row, end_row)

    # Convert the list of dictionaries to a dictionary for faster lookups
    image_dict = {item['name']: item for item in image_paths}

    # Read the existing HTML file
    with open(html_filepath, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Find all rows in the HTML file
    rows = soup.find_all('div', class_='row')

    for row in rows:
        # Get the object name for this row
        name = row.find('div', class_='name').text.strip()

        # Find the matching data from image_dict
        matching_data = image_dict.get(name)

        if matching_data:
            # Get all image containers in this row
            image_containers = row.find_all('div', class_='image-container')
            for i, img_container in enumerate(image_containers):
                # Replace the placeholder image
                img_tag = img_container.find('img')
                img_tag['src'] = matching_data['images'][i]

                # Replace the placeholder delta_t if it's not the last image
                if i < 3:
                    delta_t_tag = img_container.find('p', class_='delta-t')
                    delta_t_tag.string = f"{matching_data['delta_ts'][i]} days" if matching_data['delta_ts'][i] != 'NA' else 'NA'

    # Save the updated HTML content back to the file
    with open(html_filepath, 'w') as file:
        file.write(str(soup))

    print(f"HTML page updated and saved as {html_filepath}")

    

In [96]:
from bs4 import BeautifulSoup

def update_html_page(start_row, end_row, html_filepath='what.html'):
    # Generate the actual image data
    image_paths = get_multiple_rows(start_row, end_row)

    # Convert the list of dictionaries to a dictionary for faster lookups
    image_dict = {item['name']: item for item in image_paths}

    # Read the existing HTML file
    with open(html_filepath, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Find all rows in the HTML file
    rows = soup.find_all('div', class_='row')

    for row in rows:
        # Get the object name for this row
        name = row.find('div', class_='name').text.strip()

        # Find the matching data from image_dict
        matching_data = image_dict.get(name)

        if matching_data:
            # Get all image containers in this row
            image_containers = row.find_all('div', class_='image-container')
            for i, img_container in enumerate(image_containers):
                # Replace the placeholder image
                img_tag = img_container.find('img')
                img_tag['src'] = matching_data['images'][i]

                # Replace the placeholder delta_t if it's not the last image
                if i < 3:
                    delta_t_tag = img_container.find('p', class_='delta-t')
                    print(f"Updating delta_t for image {i}: {matching_data['delta_ts'][i]}")
                    delta_t_tag.string = f"{matching_data['delta_ts'][i]} days" if matching_data['delta_ts'][i] != 'NA' else 'NA'

    # Save the updated HTML content back to the file
    with open(html_filepath, 'w') as file:
        file.write(str(soup))

    print(f"HTML page updated and saved as {html_filepath}")


In [97]:
update_html_page(1,1)

Running for ZTF17aaapufz
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (188.96783333, 27.93205556)>
Date: None

Looking for tile observation for T17t17
Tile Found:
T17t17 VLASS1.1v2
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T17t17/
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T17t17/VLASS1.1.ql.T17t17.J123340+273000.10.2048.v1/VLASS1.1.ql.T17t17.J123340+273000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF17aaapufz_VLASS1.1v2.png already exists. Skipping download.
Run search completed in 0.33 seconds.

Looking for tile observation for T17t17
Tile Found:
T17t17 VLASS2.1
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T17t17/
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T17t17/VLASS2.1.ql.T17t17.J123340+273000.10.2048.v1/VLASS2.1.ql.T17t17.J123340+273000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\ZTF17aaapufz_VLASS2.1.png already exists. Skipping download.
Run search completed in 0.31 seconds.

Looking for

In [177]:
import vlass_search

maxi_ra = "12h10m01.32s"
maxi_dec = "+49d56m47.006s"
maxi = SkyCoord(ra = maxi_ra, dec = maxi_dec)

vlass_search.run_search("MAXI", maxi)


Running for MAXI
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (182.5055, 49.94639056)>
Date: None

Looking for tile observation for T23t13
Tile Found:
T23t13 VLASS1.1v2
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T23t13/
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T23t13/VLASS1.1.ql.T23t13.J120908+493000.10.2048.v1/VLASS1.1.ql.T23t13.J120908+493000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\MAXI_VLASS1.1v2.png already exists. Skipping download.
Run search completed in 0.25 seconds.

Looking for tile observation for T23t13
Tile Found:
T23t13 VLASS2.1
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T23t13/
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T23t13/VLASS2.1.ql.T23t13.J120908+493000.10.2048.v1/VLASS2.1.ql.T23t13.J120908+493000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
PNG file images\MAXI_VLASS2.1.png already exists. Skipping download.
Run search completed in 0.26 seconds.

Looking for tile observation for T23t13

(['images\\MAXI_VLASS1.1v2.png',
  'images\\MAXI_VLASS2.1.png',
  'images\\MAXI_VLASS3.1.png'],
 ['VLASS1.1v2', 'VLASS2.1', 'VLASS3.1'],
 [<Time object: scale='utc' format='iso' value=2017-11-20 00:00:00.000>,
  <Time object: scale='utc' format='iso' value=2020-08-01 00:00:00.000>,
  <Time object: scale='utc' format='iso' value=2023-02-04 00:00:00.000>])

In [16]:
import importlib
importlib.reload(vlass_search)

File downloaded to: VLASS_dyn_summary.php


<module 'vlass_search' from 'd:\\summer_research_code\\Query_VLASS_2\\vlass_search.py'>